# Desarollo Problemas Grafos

## Problema 2

Prestamos

### Entradas, Salidas, Precondición, Postcondición

| E/S | Nombre | Tipo | Descripción |
|-----|--------|------|-------------|
| E   |   G    |  ?   | Grafo que representa las relaciones de préstamos entre las entidades bancarias |
| S   |   b    | **boolean** | Representa si existe algun autoprestamo de alguna entidad |

### Estructura de grafo a utilizar

En este problema tendremos un grafo que tendrá los siguientes elementos:

* Nodos o Vertices: Representan las diferentes entidades bancarias que entran a ser estudiadas en el problema.
* Arcos o Aristas: Representan si existe un prestamo desde una entidad bancaria $A$ a una entidad bancaria $B$, su costo representa el valor del prestamo (Costo 0 representa que no hay prestamo).
* Dirigido: El grafo a representar es dirigido ya que el sentido de un prestamo es importante (No es lo mismo que $A$ le preste a $B$ que $B$ le preste a $A$)

### Diseño del Algoritmo a utilizar

El algoritmo que se diseñara determinara si existen algun bucle en el grafo en el que hubo prestamos. Un bucle representa que hubo cierta cantidad de dinero que se presto desde $A$ y que volvio a $A$ pasando por 0 o mas nodos distintos